# Steam 200k Cleaning

**Objective:** Clean the `steam_200k` dataset so that its `game_title` values
match with the values in the `name` column of the `games` dataset.

## Setup

In [ ]:
# Import modules
import re
import pandas as pd
import numpy as np
from utils import display_df_info
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import normalize

In [3]:
# Load playtime data
def load_playtime_data() -> pd.DataFrame:
    df = pd.read_csv('../data/steam-200k-labelled.csv')
    df.query("behavior_name == 'play'", inplace=True)
    df = df.groupby(['user_id', 'game_title'], as_index=False)[['hours']].sum()
    return df


playtime = load_playtime_data()
display_df_info(playtime, 'Playtime')

### Playtime

,Dtype,Null Count,Total,% Null
user_id,int64,0,"70,477",0.0%
game_title,object,0,"70,477",0.0%
hours,float64,0,"70,477",0.0%


,user_id,game_title,hours
0,5250,Alien Swarm,4.9
1,5250,Cities Skylines,144.0
2,5250,Deus Ex Human Revolution,62.0
3,5250,Dota 2,0.2
4,5250,Portal 2,13.6


In [4]:
# Load Games data
def load_games_data() -> pd.DataFrame:
    df = pd.read_csv('../data/game_ids.csv')
    df.rename(columns={'name': 'game_title'}, inplace=True)
    return df


games = load_games_data()
display_df_info(games, 'Games')

### Games

,Dtype,Null Count,Total,% Null
app_id,int64,0,"140,082",0.0%
game_title,object,0,"140,082",0.0%


,app_id,game_title
0,10,Counter-Strike
1,20,Team Fortress Classic
2,30,Day of Defeat
3,40,Deathmatch Classic
4,50,Half-Life: Opposing Force


## Normalize the text

In [7]:
def normalize_title(title: str) -> str:
    title = title.lower()
    title = re.sub(r'[^\w\s]', '', title)
    title = re.sub(r'\s+', ' ', title).strip()
    return title


playtime['norm_title'] = playtime['game_title'].apply(normalize_title)
display_df_info(playtime, 'Playtime', nulls=False)

games['norm_title'] = games['game_title'].apply(normalize_title)
display_df_info(games, 'Games', nulls=False)

### Playtime

,user_id,game_title,hours,norm_title
0,5250,Alien Swarm,4.9,alien swarm
1,5250,Cities Skylines,144.0,cities skylines
2,5250,Deus Ex Human Revolution,62.0,deus ex human revolution
3,5250,Dota 2,0.2,dota 2
4,5250,Portal 2,13.6,portal 2


### Games

,app_id,game_title,norm_title
0,10,Counter-Strike,counterstrike
1,20,Team Fortress Classic,team fortress classic
2,30,Day of Defeat,day of defeat
3,40,Deathmatch Classic,deathmatch classic
4,50,Half-Life: Opposing Force,halflife opposing force


In [37]:
# Exact match
matched_df = playtime.merge(games.drop('game_title', axis=1), how='left')
display_df_info(matched_df, 'Normalized Matches')

### Normalized Matches

,Dtype,Null Count,Total,% Null
user_id,int64,0,"71,046",0.0%
game_title,object,0,"71,046",0.0%
hours,float64,0,"71,046",0.0%
norm_title,object,0,"71,046",0.0%
app_id,float64,"18,457","71,046",26.0%


,user_id,game_title,hours,norm_title,app_id
0,5250,Alien Swarm,4.9,alien swarm,630.0
1,5250,Cities Skylines,144.0,cities skylines,255710.0
2,5250,Deus Ex Human Revolution,62.0,deus ex human revolution,NaN
3,5250,Dota 2,0.2,dota 2,570.0
4,5250,Portal 2,13.6,portal 2,620.0


## Pre-Model Data Preparation

In [38]:
# Prepare unmatched
unmatched = matched_df[matched_df['app_id'].isna()].copy()
display_df_info(unmatched, 'Unmatched Data', nulls=False)

name_to_app_id = dict(zip(games['norm_title'], games['app_id']))
print('Name to App ID Map')
display(list(name_to_app_id.items())[:5])

### Unmatched Data

,user_id,game_title,hours,norm_title,app_id
2,5250,Deus Ex Human Revolution,62.0,deus ex human revolution,NaN
6,76767,Age of Empires II HD Edition,13.1,age of empires ii hd edition,NaN
10,76767,Call of Duty Black Ops - Multiplayer,12.5,call of duty black ops multiplayer,NaN
11,76767,Call of Duty Modern Warfare 2,65.0,call of duty modern warfare 2,NaN
12,76767,Call of Duty Modern Warfare 2 - Multiplayer,165.0,call of duty modern warfare 2 multiplayer,NaN


Name to App ID Map


[('counterstrike', 10),
 ('team fortress classic', 20),
 ('day of defeat', 30),
 ('deathmatch classic', 40),
 ('halflife opposing force', 50)]

## Vectorization

In [ ]:
# Apply TF-IDF Vectorization
vectorizer = TfidfVectorizer(analyzer='char_wb',
                             ngram_range=(3, 5),
                             max_features=50000)
tfidf_games = vectorizer.fit_transform(games['norm_title'])
tfidf_unmatched = vectorizer.transform(unmatched['norm_title'])


# Normalize vectors to use cosine similarity via dot product
tfidf_games = normalize(tfidf_games, axis=1)
tfidf_unmatched = normalize(tfidf_unmatched, axis=1)

## Nearest Neighbors

In [40]:
nn = NearestNeighbors(n_neighbors=1, metric='cosine', algorithm='brute')
nn.fit(tfidf_games)
nn

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=1)

In [41]:
# Match in safe batches
n_unmatched = tfidf_unmatched.shape[0]
batch_size = 1000
matches = []
scores = []

for i in range(0, n_unmatched, batch_size):
    batch = tfidf_unmatched[i:i + batch_size]
    dist, idx = nn.kneighbors(batch)
    sim = 1 - dist.flatten()
    matched_names = games['norm_title'].values[idx.flatten()]
    
    matches.extend(matched_names)
    scores.extend(sim)

In [42]:
# Assign results
unmatched['fuzzy_match'] = matches
unmatched['fuzzy_score'] = scores
unmatched['fuzzy_app_id'] = unmatched['fuzzy_match'].map(name_to_app_id)

display_df_info(unmatched, 'Updated Unmatched')

### Updated Unmatched

,Dtype,Null Count,Total,% Null
user_id,int64,0,"18,457",0.0%
game_title,object,0,"18,457",0.0%
hours,float64,0,"18,457",0.0%
norm_title,object,0,"18,457",0.0%
app_id,float64,"18,457","18,457",100.0%
fuzzy_match,object,0,"18,457",0.0%
fuzzy_score,float64,0,"18,457",0.0%
fuzzy_app_id,int64,0,"18,457",0.0%


,user_id,game_title,hours,norm_title,app_id,fuzzy_match,fuzzy_score,fuzzy_app_id
2,5250,Deus Ex Human Revolution,62.0,deus ex human revolution,NaN,deus ex human revolution directors cut,0.807372,238010
6,76767,Age of Empires II HD Edition,13.1,age of empires ii hd edition,NaN,age of empires ii definitive edition,0.713221,813780
10,76767,Call of Duty Black Ops - Multiplayer,12.5,call of duty black ops multiplayer,NaN,call of duty black ops,0.737219,42700
11,76767,Call of Duty Modern Warfare 2,65.0,call of duty modern warfare 2,NaN,call of duty modern warfare,0.995571,2000950
12,76767,Call of Duty Modern Warfare 2 - Multiplayer,165.0,call of duty modern warfare 2 multiplayer,NaN,call of duty modern warfare,0.801288,2000950


In [44]:
# Update App ID if score is above threshold (75%)
threshold = 0.75
unmatched['app_id'] = np.where(unmatched['fuzzy_score'] >= threshold,
                               unmatched['fuzzy_app_id'],
                               None)
unmatched.head()

,user_id,game_title,hours,norm_title,app_id,fuzzy_match,fuzzy_score,fuzzy_app_id
2,5250,Deus Ex Human Revolution,62.0,deus ex human revolution,238010,deus ex human revolution directors cut,0.807372,238010
6,76767,Age of Empires II HD Edition,13.1,age of empires ii hd edition,None,age of empires ii definitive edition,0.713221,813780
10,76767,Call of Duty Black Ops - Multiplayer,12.5,call of duty black ops multiplayer,None,call of duty black ops,0.737219,42700
11,76767,Call of Duty Modern Warfare 2,65.0,call of duty modern warfare 2,2000950,call of duty modern warfare,0.995571,2000950
12,76767,Call of Duty Modern Warfare 2 - Multiplayer,165.0,call of duty modern warfare 2 multiplayer,2000950,call of duty modern warfare,0.801288,2000950


In [45]:
# Merge into final
matched_df['app_id'] = matched_df['app_id'].fillna(unmatched['app_id'])
display_df_info(matched_df, 'New Matched Data')

C:\Users\msb\AppData\Local\Temp\ipykernel_72440\302085564.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  matched_df['app_id'] = matched_df['app_id'].fillna(unmatched['app_id'])


### New Matched Data

,Dtype,Null Count,Total,% Null
user_id,int64,0,"71,046",0.0%
game_title,object,0,"71,046",0.0%
hours,float64,0,"71,046",0.0%
norm_title,object,0,"71,046",0.0%
app_id,float64,"7,555","71,046",10.6%


,user_id,game_title,hours,norm_title,app_id
0,5250,Alien Swarm,4.9,alien swarm,630.0
1,5250,Cities Skylines,144.0,cities skylines,255710.0
2,5250,Deus Ex Human Revolution,62.0,deus ex human revolution,238010.0
3,5250,Dota 2,0.2,dota 2,570.0
4,5250,Portal 2,13.6,portal 2,620.0


In [46]:
# Check results
coverage = matched_df['app_id'].notnull().mean()
print(f"Final app_id match coverage: {coverage:.2%}")

Final app_id match coverage: 89.37%


In [ ]:
output_df = matched_df.copy()

output_df['app_id'] = output_df['app_id'].fillna(-1).astype(int)
output_df['app_id'] = np.where(output_df['app_id'] == -1,
                               None,
                               output_df['app_id'])

output_df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71046 entries, 0 to 71045
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   user_id     71046 non-null  int64  
 1   game_title  71046 non-null  object 
 2   hours       71046 non-null  float64
 3   norm_title  71046 non-null  object 
 4   app_id      63491 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 2.7+ MB


,user_id,game_title,hours,norm_title,app_id
0,5250,Alien Swarm,4.9,alien swarm,630
1,5250,Cities Skylines,144.0,cities skylines,255710
2,5250,Deus Ex Human Revolution,62.0,deus ex human revolution,238010
3,5250,Dota 2,0.2,dota 2,570
4,5250,Portal 2,13.6,portal 2,620
5,5250,Team Fortress 2,0.8,team fortress 2,440
6,76767,Age of Empires II HD Edition,13.1,age of empires ii hd edition,None
7,76767,Alien Swarm,0.8,alien swarm,630
8,76767,Banished,24.0,banished,242920
9,76767,Call of Duty Black Ops,22.0,call of duty black ops,42700


In [58]:
output_df.to_csv('../data/playtime-cleaned.csv', index=False)